# Task4 分组
## 1 知识梳理（重点记忆）

### 1.1 分组模式及其对象
- 每个组可以返回一个标量值
- 每个组可以返回一个Series类型
- 可以返回整个组所在行，该类型是`DataFrame`类型

### 1.2 agg方法
- 将多个聚合函数，采用列表的形式传参
- 使用字典的方式，针对不同的列进行聚合统计
- 使用自定义函数进行计算，迭代值是每列
- 通过元组进行索引列的重命名

### 1.3 变换和过滤
#### 1.3.1 transform方法
- 使用group对象，迭代值是每列
- 可传入聚合方法对应的字符串

### 1.3.2 组索引与过滤
使用`filter`方法进行组的筛选，迭代值是每个分组的`DataFrame`对象

### 1.4 跨组分组
使用`apply`方法，传入的函数只允许返回布尔值，即条件函数

## 2 练一练

### 2.1 第1题

请根据上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值。

**我的解答：**

In [1]:
import pandas as pd

df = pd.read_csv('../data/learn_pandas.csv')

In [2]:
low_condition = df.Weight < df.Weight.quantile(0.25)
df.groupby(low_condition)['Height'].mean()

Weight
False    165.950704
True     153.753659
Name: Height, dtype: float64

In [3]:
normal_condition = (df.Weight.quantile(0.25) < df.Weight) & (df.Weight < df.Weight.quantile(0.75))
df.groupby(normal_condition)['Height'].mean()

Weight
False    164.084000
True     162.174699
Name: Height, dtype: float64

In [4]:
high_condition = df.Weight > df.Weight.quantile(0.75)
df.groupby(high_condition)['Height'].mean()

Weight
False    159.727660
True     174.935714
Name: Height, dtype: float64

### 2.2 第2题
上一小节介绍了可以通过`drop_duplicates`得到具体的组类别，现请用`groups`属性完成类似的功能。

In [5]:
df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,Shanghai Jiao Tong University,Female
1,Peking University,Male
2,Shanghai Jiao Tong University,Male
3,Fudan University,Female
4,Fudan University,Male
5,Tsinghua University,Female
9,Peking University,Female
16,Tsinghua University,Male


**我的解答：**

In [6]:
groups = df.groupby(['School', 'Gender']).groups
school_list, gender_list, index_list = [], [], []
for key, value in groups.items():
    index_list.append(value[0])
    school_list.append(key[0])
    gender_list.append(key[1])

pd.DataFrame(data={'School':school_list, 'Gender':gender_list}, index=index_list).sort_index()  

,School,Gender
0,Shanghai Jiao Tong University,Female
1,Peking University,Male
2,Shanghai Jiao Tong University,Male
3,Fudan University,Female
4,Fudan University,Male
5,Tsinghua University,Female
9,Peking University,Female
16,Tsinghua University,Male


### 2.3 第3题
请查阅文档，明确`all/any/mad/skew/sem/prod`函数的含义。

**我的解答：**

In [7]:
import numpy as np

np.random.seed(0)
df_ex3 = pd.DataFrame(np.random.randint(0,2,(6,4)), columns=['a', 'b', 'c', 'd'])
df_ex3['e'] = [[i, j] for (i, j) in zip(df_ex3['a'], df_ex3['b'])]
df_ex3

,a,b,c,d,e
0,0,1,1,0,"[0, 1]"
1,1,1,1,1,"[1, 1]"
2,1,1,1,0,"[1, 1]"
3,0,1,0,0,"[0, 1]"
4,0,0,0,1,"[0, 0]"
5,0,1,1,0,"[0, 1]"


In [8]:
# 如果分组中的该列的所有值（1为True，0为False）都是True，则为True，否则为False
df_ex3.groupby(['a', 'b']).all()

c      d     e
a b                    
0 0  False   True  True
  1  False  False  True
1 1   True  False  True

In [9]:
# 如果分组中的该列的任何值为（1为True，0为False）是True，则为True，否则为False
df_ex3.groupby(['a', 'b']).any()

c      d     e
a b                    
0 0  False   True  True
  1   True  False  True
1 1   True   True  True

In [10]:
# 计算各分组的平均绝对偏差
df_ex3.groupby(['a', 'b']).mad()

c    d
a b               
0 0  0.000000  0.0
  1  0.444444  0.0
1 1  0.000000  0.5

In [11]:
# 计算各分组的偏度
df_ex3.groupby(['a', 'b']).skew()

c    d
a b               
0 0       NaN  NaN
  1 -1.732051  0.0
1 1       NaN  NaN

In [12]:
# 计算各分组的平均无偏标准误差
df_ex3.groupby(['a', 'b']).sem()

c    d
a b               
0 0       NaN  NaN
  1  0.333333  0.0
1 1  0.000000  0.5

In [13]:
# 计算各分组的乘积
df_ex3.groupby(['a', 'b']).prod()

c  d
a b      
0 0  0  1
  1  0  0
1 1  1  0

### 2.4 第4题
请使用【b】中的传入字典的方法完成【a】中等价的聚合任务

In [14]:
df = pd.read_csv('../data/learn_pandas.csv')
gb = df.groupby('Gender')[['Height', 'Weight']]

# 示例【a】
gb.agg(['sum', 'idxmax', 'skew'])

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

**我的解答：**

In [15]:
methods = ['sum','idxmax', 'skew']

gb.agg({'Height':methods, 'Weight':methods})

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

### 2.5 第5题
在`groupby`对象中可以使用`describe`方法进行统计信息汇总，请同时使用多个聚合函数，完成与该方法相同的功能。

In [16]:
df = pd.read_csv('../data/learn_pandas.csv')
gb = df.groupby('Gender')[['Height', 'Weight']]

gb.describe()

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

**我的解答：**

In [17]:
def quantile(n):
    def quantile_(x):
        return x.quantile(n/100)
    quantile_.__name__ = '{0}%'.format(n)
    return quantile_

methods = ['count','mean', 'std', 'min', quantile(25), quantile(50), quantile(75), 'max']

res = gb.agg({'Height':methods, 'Weight':methods})
res = res.astype(np.float64)
res

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

### 2.6 第6题
在`groupby`对象中，`rank`方法也是一个实用的变换函数，请查阅它的功能并给出一个使用的例子。

**我的解答：**

`rank`方法主要功能是对其它列的属于同分组数据进行数值大小排序，重点参数如下表：  

|参数|描述|
|---|:---|
|method|如果值相同，采用方法进行计算排名，默认方法为average，可选方法为average、min、max、first、dense|
|ascending|默认升序为True，降序为False|
|na_option|NaN的数据位置，默认为keep（保持原位）,可选值为keep、top、bottom|
|pct|计算分组数据的百分比值|
|axis|默认为0，0为列，1为行|

In [18]:
np.random.seed(0)
df_ex6 = pd.DataFrame(np.random.randint(0,6,(6,4)), columns=['a', 'b', 'c', 'd'])
df_ex6

,a,b,c,d
0,4,5,0,3
1,3,3,1,3
2,5,2,4,0
3,0,4,2,1
4,0,1,5,1
5,5,0,1,4


In [19]:
gb = df_ex6.groupby('a')

In [20]:
gb.rank(method='first')

,b,c,d
0,1.0,1.0,1.0
1,1.0,1.0,1.0
2,2.0,2.0,1.0
3,2.0,1.0,1.0
4,1.0,2.0,2.0
5,1.0,1.0,2.0


### 2.7 第7题
对于`transform`方法无法像`agg`一样，通过传入字典来对指定列使用特定的变换，如果需要在一次`transform`的调用中实现这种功能，请给出解决方案。

**我的解答：**

In [21]:
df = pd.read_csv('../data/learn_pandas.csv')
gb = df.groupby('Gender')[['Height', 'Weight']]

In [22]:
def my_func(method_dict):
    def my_(x):
        method = method_dict[x.name]
        if method == 'standardized':
            return (x-x.mean())/x.std()
        elif method == 'max':
            return x.max()
    return my_

gb.transform(my_func({'Height':'standardized', 'Weight':'max'})).head()

,Height,Weight
0,-0.058760,63.0
1,-1.010925,89.0
2,2.167063,89.0
3,NaN,63.0
4,0.053133,89.0


### 2.8 第8题
从概念上说，索引功能是组过滤功能的子集，请使用`filter`函数完成`loc[...]`的功能，这里假设"`...`"是元素列表。 

In [23]:
df = pd.read_csv('../data/learn_pandas.csv', usecols = ['School', 'Grade', 'Name', 'Gender', 'Weight', 'Transfer'])
df_demo = df.set_index('Name')
df_demo.loc[['Qiang Sun','Quan Zhao'], ['School','Gender']]

,School,Gender
Name,,
Qiang Sun,Tsinghua University,Female
Qiang Sun,Tsinghua University,Female
Qiang Sun,Shanghai Jiao Tong University,Female
Quan Zhao,Shanghai Jiao Tong University,Female


**我的解答：**

In [24]:
names, cols= ['Qiang Sun','Quan Zhao'], ['School','Gender']

def my_filter(df, names, cols):
    gb = df.groupby('Name')[cols]
    name_indexs = []
    for name in names:
        name_indexs += gb.get_group(name).index.tolist()    
    
    res = gb.filter(lambda x : len(set(x.index.values.tolist()) & set(name_indexs)) > 0)
    return res
res = my_filter(df, names, cols)
res

,School,Gender
27,Tsinghua University,Female
106,Tsinghua University,Female
122,Shanghai Jiao Tong University,Female
172,Shanghai Jiao Tong University,Female


### 2.9 第9题
请尝试在`apply`传入的自定义函数中，根据组的某些特征返回相同长度但索引不同的`Series`，会报错吗？

**我的解答：**

In [25]:
df = pd.read_csv('../data/learn_pandas.csv')
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]
gb.count()

Height  Weight
Gender Test_Number                
Female 1                63      64
       2                49      52
       3                20      19
Male   1                27      29
       2                18      18
       3                 6       7

In [26]:
def my_func_ex9(x):
    if x.shape[0] < 100:
        return pd.Series([0,0], index=['a', 'b'])
    else:
        return pd.Series([0,0], index=['b', 'c'])

try:
    gb.apply(my_func_ex9)
except Exception as e:
    Exception_Msg = e
    print("Exception Message:", Exception_Msg)    

执行函数之后可知，返回相同长度但索引不同的`Series`会报错，原因是`Series`的`name`必须是可哈希的（不可变的），但是代码中通过强制返回索引不同的`Series`名字，导致报错。

### 2.10 第10题
请尝试在`apply`传入的自定义函数中，根据组的某些特征返回相同大小但列索引不同的`DataFrame`，会报错吗？如果只是行索引不同，会报错吗？

**我的解答：**

In [27]:
df = pd.read_csv('../data/learn_pandas.csv')
gb = df.groupby(['Gender'])[['Height','Weight']]
gb.count()

,Height,Weight
Gender,,
Female,132,135
Male,51,54


In [28]:
# 返回相同大小但列索引不同的DataFrame
def my_func_ex10(x):
    if x.shape[0] > 100:
        return pd.DataFrame(np.ones((2,2)), index = ['a','b'], columns=pd.Index([('w','x'),('y','z')]))
    else:
        return pd.DataFrame(np.ones((2,2)), index = ['a','b'], columns=pd.Index([('w','d'),('y','f')]))
        
try:
    gb.apply(my_func_ex10)
except Exception as e:
    Exception_Msg = e
    print("Exception Message:", Exception_Msg)

In [29]:
gb.apply(my_func_ex10)

w         y     
            d    x    f    z
Gender                      
Female a  NaN  1.0  NaN  1.0
       b  NaN  1.0  NaN  1.0
Male   a  1.0  NaN  1.0  NaN
       b  1.0  NaN  1.0  NaN

In [30]:
# 返回相同大小但行索引不同的DataFrame
def my_func_ex10(x):
    if x.shape[0] > 100:
        return pd.DataFrame(np.ones((2,2)), index = ['a','b'], columns=pd.Index([('w','x'),('y','z')]))
    else:
        return pd.DataFrame(np.ones((2,2)), index = ['g','h'], columns=pd.Index([('w','d'),('y','f')]))
        
try:
    gb.apply(my_func_ex10)
except Exception as e:
    Exception_Msg = e
    print("Exception Message:", Exception_Msg)

In [31]:
gb.apply(my_func_ex10)

w         y     
            d    x    f    z
Gender                      
Female a  NaN  1.0  NaN  1.0
       b  NaN  1.0  NaN  1.0
Male   g  1.0  NaN  1.0  NaN
       h  1.0  NaN  1.0  NaN

观察得知，程序执行不会报错

### 2.11 第11题
在`groupby`对象中还定义了`cov`和`corr`函数，从概念上说也属于跨列的分组处理。请利用之前定义的`gb`对象，使用apply函数实现与`gb.cov()`同样的功能并比较它们的性能。

In [32]:
df = pd.read_csv('../data/learn_pandas.csv')
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]
gb.cov()

Height     Weight
Gender Test_Number                             
Female 1           Height  20.963600  21.452034
                   Weight  21.452034  26.438244
       2           Height  31.615680  30.386170
                   Weight  30.386170  34.568250
       3           Height  23.582395  20.801307
                   Weight  20.801307  23.228070
Male   1           Height  42.638234  48.785833
                   Weight  48.785833  67.669951
       2           Height  57.041732  38.224183
                   Weight  38.224183  37.869281
       3           Height  56.157667  84.020000
                   Weight  84.020000  89.904762

**我的解答：**

In [33]:
def my_cov(x):  
    cov_height_weight = x['Height'].cov(x['Weight'])
    cov_height_height = x['Height'].cov(x['Height'])
    cov_weight_weight = x['Weight'].cov(x['Weight'])
    
    return pd.DataFrame([[cov_height_height, cov_height_weight], [cov_height_weight, cov_weight_weight]], 
                        index = ['Height','Weight'], columns = pd.Index(['Height', 'Weight']))

gb.apply(my_cov)

Height     Weight
Gender Test_Number                             
Female 1           Height  20.963600  21.452034
                   Weight  21.452034  26.438244
       2           Height  31.615680  30.386170
                   Weight  30.386170  34.568250
       3           Height  23.582395  20.801307
                   Weight  20.801307  23.228070
Male   1           Height  42.638234  48.785833
                   Weight  48.785833  67.669951
       2           Height  57.041732  38.224183
                   Weight  38.224183  37.869281
       3           Height  56.157667  84.020000
                   Weight  84.020000  89.904762

性能测试对比：

In [34]:
%timeit -n 30 gb.cov()

30 loops, best of 5: 6.62 ms per loop


In [35]:
%timeit -n 30 gb.apply(my_cov)

30 loops, best of 5: 13.7 ms per loop


性能测试分析：my_cov的程序比groupby.cov()慢了一倍，应该是计算Series.cov()时间慢

## 3 练习

### 3.1 Ex1：汽车数据集
现有一份汽车数据集，其中`Brand, Disp., HP`分别代表汽车品牌、发动机蓄量、发动机输出。

In [36]:
df = pd.read_csv('../data/car.csv')
df.head(3)

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63


1. 先过滤出所属`Country`数超过2个的汽车，即若该汽车的`Country`在总体数据集中出现次数不超过2则剔除，再按`Country`分组计算价格均值、价格变异系数、该`Country`的汽车数量，其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为`CoV`。
2. 按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计`Price`的均值。
3. 对类型`Type`分组，对`Price`和`HP`分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单层索引。
4. 对类型`Type`分组，对`HP`进行组内的`min-max`归一化。
5. 对类型`Type`分组，计算`Disp.`与`HP`的相关系数。

**我的解答：**

**第1问：**

In [37]:
# 过滤出所属Country数超过2个的汽车
df_1 = df.groupby('Country').filter(lambda x: x.shape[0] > 2)
df_1.head()

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63
3,Honda Civic 4,6635,Japan/USA,5.0,32,Small,2260,91,92
4,Mazda Protege 4,6599,Japan,5.0,32,Small,2440,113,103


In [38]:
df_1.groupby('Country')['Price'].agg(['mean', ('CoV', lambda x : x.std()/x.mean()), 'count'])

,mean,CoV,count
Country,,,
Japan,13938.052632,0.387429,19
Japan/USA,10067.571429,0.240040,7
Korea,7857.333333,0.243435,3
USA,12543.269231,0.203344,26


**第2问：**

In [39]:
df_size = df.shape[0]
condition = ['head'] * int(df_size / 3) + ['mid'] * int(df_size / 3) + ['tail'] * int(df_size / 3)

df.groupby(condition)['Price'].mean()

head     9069.95
mid     13356.40
tail    15420.65
Name: Price, dtype: float64

**第3问：**

In [40]:
df_2 = df.groupby('Type').agg({'Price':['max'], 'HP':['min']})
df_2

,Price,HP
,max,min
Type,,
Compact,18900,95
Large,17257,150
Medium,24760,110
Small,9995,63
Sporty,13945,92
Van,15395,106


In [41]:
df_2.columns = df_2.columns.map(lambda x : '_'.join(x))
df_2

,Price_max,HP_min
Type,,
Compact,18900,95
Large,17257,150
Medium,24760,110
Small,9995,63
Sporty,13945,92
Van,15395,106


**第4问：对类型`Type`分组，对HP进行组内的`min-max`归一化。**

In [42]:
df.groupby('Type')['HP'].transform(lambda x: (x-x.min())/(x.max() - x.min())).head()

0    1.00
1    0.54
2    0.00
3    0.58
4    0.80
Name: HP, dtype: float64

**第5问：对类型Type分组，计算`Disp.`与`HP`的相关系数**

In [43]:
gb = df.groupby('Type')[['Disp.', 'HP']]

gb.apply(lambda x: np.corrcoef(x['Disp.'], x['HP'])[0,1])

Type
Compact    0.586087
Large     -0.242765
Medium     0.370491
Small      0.603916
Sporty     0.871426
Van        0.819881
dtype: float64

### 3.2 Ex2：实现transform函数
* `groupby`对象的构造方法是`my_groupby(df, group_cols)`
* 支持单列分组与多列分组
* 支持带有标量广播的`my_groupby(df)[col].transform(my_func)`功能
* `pandas`的`transform`不能跨列计算，请支持此功能，即仍返回`Series`但`col`参数为多列
* 无需考虑性能与异常处理，只需实现上述功能，在给出测试样例的同时与`pandas`中的`transform`对比结果是否一致

**解题思路：**